<a href="https://colab.research.google.com/github/caoTayTang/image_super_resolution_traditional_cv/blob/main/notebooks/experiments_ml_patch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("Test ML Patch")